In [1]:
import numpy as np
import sympy as sp
from scipy.linalg import lu 

Three linear eq.s</br>
<math xmlns="http://www.w3.org/1998/Math/MathML">
  <mtable columnalign="right left right left right left right left right left right left" rowspacing="3pt" columnspacing="0em 2em 0em 2em 0em 2em 0em 2em 0em 2em 0em" displaystyle="true">
    <mtr>
      <mtd>
        <mi>x</mi>
        <mo>&#x2212;<!-- − --></mo>
        <mn>2</mn>
        <mi>y</mi>
        <mo>+</mo>
        <mn>3</mn>
        <mi>z</mi>
        <mo>=</mo>
        <mn>9</mn>
      </mtd>
      <mtd />
      <mtd>
        <mtext>...(1)</mtext>
      </mtd>
    </mtr>
    <mtr>
      <mtd>
        <mo>&#x2212;<!-- − --></mo>
        <mi>x</mi>
        <mo>+</mo>
        <mn>3</mn>
        <mi>y</mi>
        <mo>&#x2212;<!-- − --></mo>
        <mi>z</mi>
        <mo>=</mo>
        <mo>&#x2212;<!-- − --></mo>
        <mn>6</mn>
      </mtd>
      <mtd />
      <mtd>
        <mtext>...(2)</mtext>
      </mtd>
    </mtr>
    <mtr>
      <mtd>
        <mn>2</mn>
        <mi>x</mi>
        <mo>&#x2212;<!-- − --></mo>
        <mn>5</mn>
        <mi>y</mi>
        <mo>+</mo>
        <mn>5</mn>
        <mi>z</mi>
        <mo>=</mo>
        <mn>17</mn>
      </mtd>
      <mtd />
      <mtd>
        <mtext>...(3)</mtext>
      </mtd>
    </mtr>
  </mtable>
</math>

In [2]:
M= np.array([ [1,-2,3], [-1,3,-1], [2,-5,5] ]) 
M

array([[ 1, -2,  3],
       [-1,  3, -1],
       [ 2, -5,  5]])

In [3]:
b= np.array([ [9,-6,17] ]) 
b.T

array([[ 9],
       [-6],
       [17]])

In [4]:
P, L, U = lu(M)
print('P:\n',P)
print('L:\n',L)
print('U:\n',U)

P:
 [[0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
L:
 [[ 1.   0.   0. ]
 [-0.5  1.   0. ]
 [ 0.5  1.   1. ]]
U:
 [[ 2.  -5.   5. ]
 [ 0.   0.5  1.5]
 [ 0.   0.  -1. ]]


https://en.wikipedia.org/wiki/LU_decomposition#Solving_linear_equations

In [5]:
Pb = P @ b.T 
Pb

array([[17.],
       [-6.],
       [ 9.]])

In [6]:
x,y,z = sp.symbols([ 'x', 'y', 'z' ]) #actual variables
p,q,r = sp.symbols([ 'p', 'q', 'r' ]) #pseudo variables represent y ...see WIKI

In [7]:
actual_var=np.array([ [x,y,z] ])
pseudo_var=np.array([ [p,q,r] ])

In [8]:
Ly=L @ pseudo_var.T
print('Ly = Pb ..in an elementwise manner')

eq_pqr=[]
for i in range(len(Ly)):
    eq_pqr.append(sp.Eq(Ly[i,0], Pb[i,0]))
    
print(eq_pqr)

Ly = Pb ..in an elementwise manner
[Eq(1.0*p, 17.0), Eq(-0.5*p + 1.0*q, -6.0), Eq(0.5*p + 1.0*q + 1.0*r, 9.0)]


## Forward Substitution Begins... 

In [9]:
pval= sp.solve(eq_pqr[0])
pval

[17.0000000000000]

In [10]:
qval= sp.solve( eq_pqr[1].evalf(subs={p:pval[0]}) )
qval

[2.50000000000000]

In [11]:
rval= sp.solve( eq_pqr[2].evalf(subs={p:pval[0], q:qval[0]}) )
rval

[-2.00000000000000]

## First Step Completed

In [12]:
_y= [pval[0], qval[0], rval[0]]  #recast as y for known p,q,r to keep in line with WIKI syntax
_y

[17.0000000000000, 2.50000000000000, -2.00000000000000]

In [13]:
Ux= U @ actual_var.T
Ux

array([[2.0*x - 5.0*y + 5.0*z],
       [0.5*y + 1.5*z],
       [-1.0*z]], dtype=object)

In [14]:
print('Ux = y  ...elementwise')
eq_xyz=[]
for i in range(len(Ux)):
    eq_xyz.append(sp.Eq(Ux[i,0], _y[i]))
    
print(eq_xyz)

Ux = y  ...elementwise
[Eq(2.0*x - 5.0*y + 5.0*z, 17.0), Eq(0.5*y + 1.5*z, 2.5), Eq(-1.0*z, -2.0)]


## Backward Substitution Begins... 

In [15]:
zval=sp.solve(eq_xyz[2])
zval

[2.00000000000000]

In [16]:
yval= sp.solve( eq_xyz[1].evalf(subs={z:zval[0]}) )
yval

[-1.00000000000000]

In [17]:
xval= sp.solve( eq_xyz[0].evalf(subs={z:zval[0], y:yval[0]}) )
xval

[1.00000000000000]

In [18]:
xyz_var= np.array([ [xval[0], yval[0], zval[0] ] ])
xyz_var.T

array([[1.00000000000000],
       [-1.00000000000000],
       [2.00000000000000]], dtype=object)

In [19]:
(M @ xyz_var.T) - b.T == np.zeros((3,1))

array([[ True],
       [ True],
       [ True]])

In [20]:
np.allclose(M @ xyz_var.T, b.T)

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''